In [1]:
from anomalib.data import PredictDataset
from anomalib.engine import Engine
import torch
from anomalib.models import Patchcore
from safetensors.torch import load_file
from torchvision import models
import os
from pytorch_lightning import seed_everything
from anomalib.data import Folder

torch.cuda.empty_cache()
torch.set_float32_matmul_precision('medium')
seed_everything(42, workers=True)

weights = torch.load("/wrk/weights/resnet50.pth", map_location='cuda')
custom_backbone = models.resnet50()
custom_backbone.load_state_dict(weights)

model = Patchcore(
    backbone=custom_backbone,
    layers=["layer2", "layer3"],
    pre_trained=False,
    coreset_sampling_ratio=0.1,
    num_neighbors=5
)

engine = Engine(
                accelerator='cuda', 
                enable_progress_bar=True
                )

test_dataset = Folder(
    name="custom_folder",
    root="/wrk/data/processed",
    normal_dir="normal_dir",
    abnormal_dir="abnormal_dir",
    mask_dir="mask_dir",
    train_batch_size=8,
    eval_batch_size=8,
    num_workers=2,
    seed=42    
)

predictions = engine.predict(model=model,
                             datamodule=test_dataset,
                             ckpt_path="/wrk/notebooks/results/Patchcore/custom_folder/fit_50_2layers_0.1_5/weights/lightning/model.ckpt")

for prediction in predictions:
    image_path = prediction.image_path
    anomaly_map = prediction.anomaly_map
    pred_label = prediction.pred_label
    pred_score = prediction.pred_score

/opt/conda/envs/anomalib/lib/python3.10/site-packages/timm/models/layers/__init__.py:48: FutureWarning: Importing from timm.models.layers is deprecated, please import via timm.layers
  warnings.warn(f"Importing from {__name__} is deprecated, please import via timm.layers", FutureWarning)
Seed set to 42
/opt/conda/envs/anomalib/lib/python3.10/site-packages/lightning/pytorch/utilities/parsing.py:210: Attribute 'backbone' is an instance of `nn.Module` and is already saved during checkpointing. It is recommended to ignore them using `self.save_hyperparameters(ignore=['backbone'])`.
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
Restoring states from the checkpoint path at /wrk/notebooks/results/Patchcore/custom_folder/fit_50_2layers_0.1_5/weights/lightning/model.ckpt
/opt/conda/envs/anomalib/lib/python3.10/site-packages/lightning/pytorch/callbacks/model_checkpoint.py:445: The dirpath has changed from '/wrk/notebooks/resul

Predicting: |          | 0/? [00:00<?, ?it/s]